In [1]:
import glob
import ipywidgets as widgets
import math
import time
import plotly.graph_objects as go

from ipywidgets import interact, VBox

from editolidoHelper import (
    load_ofp_route,
    get_nearest_wmo,
    get_nearest_wmo_results,
    get_nearby_wmo,
    basemap,
    scatter_geopoints,
    wmo_grid,
    wmo_loading_time,
)




In [2]:
from editolido.geolite import rad_to_nm
def find_strategic(start, end, results, o_xtds, l=0, course_max_diff=45):
    """
    Find point you can not suppress without increasing xtd
    :param l: penalty factor
    :param o_xtds: a precalculated list of ogimet xtd
    :param start: int
    :param end: int
    :param results: [Result]
    :return:
    """
    # search in reverse order to stop at the latest point in the route direction
    # in segment [i, j] we try to remove inner elements by checking the xtd
    sign = lambda a: 1 - (a<0)
    previous_sign = sign(o_xtds[start])
    for k in range(end - 1, start, -1):
        # xtd from fpl point to ogimet segment
        f_xtd = results[k].fpl.xtd_to(
            (results[start].ogimet, results[end].ogimet)
        )
        o_xtd = o_xtds[k]
        new_sign = sign(o_xtd)
        # if abs(o_xtd) < abs(f_xtd) < results[k].fpl.distance_to(results[k+1].fpl):
        #     return k
        penalty = 0
        K = 1
        course_diff = (180/math.pi) * (
                abs(
                    (results[k - 1].ogimet.course_to(results[k].ogimet) - results[k].ogimet.course_to(results[k + 1].ogimet))
                    -
                    (results[k - 1].fpl.course_to(results[k].fpl) - results[k].fpl.course_to(results[k + 1].fpl))
                ) % math.pi
            )
        if abs(f_xtd) > abs(o_xtd):
            o_d = results[k].ogimet.distance_to(results[k-1].ogimet)
            ratio = o_xtds[k]**2 / math.sqrt(o_d)  if o_d != 0 else 0
            if l > 0 and previous_sign != new_sign:
                penalty = abs(ratio * l)
            if l >= 0:
                #print('o_xtd', rad_to_nm(o_xtds[k]), 'o_d', rad_to_nm(o_d), 'o_xtd start', rad_to_nm(o_xtds[start]), 'f_xtd', rad_to_nm(f_xtd), 'ratio', rad_to_nm(ratio))
                #print('min l to reject {0}({1}): {2}'.format(results[k].ogimet.name, results[k].fpl.name, K * (abs(f_xtd) - abs(o_xtd))/ abs(ratio)))
                if previous_sign != new_sign:
                    results[k].ogimet.description = 'min l: {0:.2f}, max course: {1:.2f}'.format(K * (abs(f_xtd) - abs(o_xtd))/ abs(ratio), course_diff)
                else:
                    results[k].ogimet.description = 'course diff: {0:.2f} {1:.3f}<{2:.3f}'.format(
                        course_diff,
                        rad_to_nm(o_xtds[k+1]),
                        rad_to_nm(o_xtds[k]))
        if (abs(f_xtd) > abs(o_xtd) + (penalty / K)) or (abs(f_xtd) > abs(o_xtd) and course_diff < course_max_diff):
            if not (course_diff > 50 and abs(o_xtds[k+1]) < abs(o_xtds[k])):
                return k
    return None


def find_best_xtd(start, end, o_xtds):
    best = None
    best_xtd = 0
    for k in range(start + 1, end):
        # xtd = results[k].fpl.xtd_to((results[k - 1].ogimet, results[k + 1].ogimet))
        xtd = o_xtds[k]
        if best is None or abs(xtd) < abs(best_xtd):
            best = k
            best_xtd = xtd
    return best


def filter_by_xtd(results, max_distance=0, l=0, recursion=0, course_max_diff=45):
    """
    Here we keep significant ogimet points.
    By significant, I mean points which increase the xtd if missing.
    The algorithm is recursive, if route is A B C D E F
    and ogimet route found is A B'C'D'E'F
    We try to suppress B', if successful we try to suppress C' and so on
    For example if B', C' and E' are not relevant the loop
    will try to suppress B' and C', then it will keep D' and
    start again from D' to suppress E' and keep F
    At the end we try again (recursion) until the route size is constant.
    For information a typical NAT route will reduce from 26 to 15 points
    and a flight to NRT will end with 26 points (starting from 79)
    :param results: [Result]
    :return: [Result]
    """
    res = [results[0]]
    i = -1
    o_xtds = [results[0].ogimet.xtd_to((results[0].fpl, results[1].fpl))]  # departure
    length = len(results)
    for k in range(1, length - 1):
        o_xtd1 = results[k].ogimet.xtd_to(
            (results[k].fpl, results[k + 1].fpl)
        )
        o_xtd0 = results[k].ogimet.xtd_to(
            (results[k - 1].fpl, results[k].fpl)
        )
        if abs(o_xtd1) < abs(o_xtd0):
            o_xtd = o_xtd1
        else:
            o_xtd = o_xtd0
        o_xtds.append(o_xtd)
    o_xtds.append(results[-1].ogimet.xtd_to((results[-2].fpl, results[-1].fpl)))  # arrival
    while i < length - 1:
        i += 1
        j = i + 2
        # we try to remove many consecutive points until it fails
        while j <= length - 1:
            d = 0
            if max_distance > 0:
                d = results[i].fpl.distance_to(results[j].fpl)
            if d <= max_distance:
                k = find_strategic(i, j, results, o_xtds, l=(l if recursion > 0 else 0), course_max_diff=course_max_diff)
                # if k:
                #     print(results[k].ogimet.pid, recursion, results[i].ogimet.pid, results[j].ogimet.pid, i, j)
            elif d > max_distance and (recursion == 0 or j > i + 2):  # j > i + 2 is to limit recursion
                k = find_best_xtd(i, j, o_xtds)
                #print(results[k].ogimet.name, recursion, results[i].ogimet.name, results[j].ogimet.name, i, j)
            else:
                #print(recursion, results[i].ogimet.pid, results[j].ogimet.pid, i, j)
                k = j - 1
            if k is None:
                j += 1  # no significant point yet, try to extend to next
            else:
                # a significant point was found, store it
                if results[k].ogimet.name not in [o.name for _, o in res]:
                    res.append(results[k])
                i = k - 1  # will start at k on next round
                break
    res.append(results[-1])
    # recursion works, so try it until there is no change
    if len(res) < len(results):
        return filter_by_xtd(res, max_distance=max_distance, l=l, recursion=recursion+1, course_max_diff=course_max_diff)
    else:
        return res

In [ ]:
from editolido.ogimet import ogimet_route, reduce_results
from editolido.geopoint import GeoPoint

@interact(file=glob.glob('ofp/*.route.json'), l=[0, 5, 10, 35, 40, 100], max_length=[0, 'route/10'], course_max_diff=[25, 30, 35, 40, 45, 60, 70])
def plot(file='ofp/AF264_24FEB21_CDG_Dossier de Vol OFP 5-0-1.route.json', l=35, max_length=0, course_max_diff=35):
    log_view = widgets.Output(layout={'border': '1px solid black', 'height': '3em'})
    route = load_ofp_route(file)
    max_distance = 0  if max_length == 0 else route.distance(converter=None)/10

    def get_new_ogimet_route():
        start_time = time.time()
        ogimet_results = get_nearest_wmo_results(route, wmo_grid)
        ogimet_results = filter_by_xtd(ogimet_results, max_distance=max_distance, l=l, course_max_diff=course_max_diff)
        ogimet_results = reduce_results(ogimet_results)
        exec_time = (time.time() - start_time) * 1000
        log_view.append_stdout('new algorithm exec time: {0:.2f}ms, {1} points'.format(exec_time, len(ogimet_results)))
        return [
            GeoPoint(
                r.ogimet,
                name=r.ogimet.name  + ((' ' + r.ogimet.description) if r.ogimet.description else ''))
            for r in ogimet_results]

    def get_old_ogimet_route():
        start_time = time.time()
        old_route = ogimet_route(route, segment_size=0)
        exec_time = (time.time() - start_time) * 1000
        log_view.append_stdout('old algorithm exec time: {0:.2f}ms, {1} points'.format(exec_time - wmo_loading_time, len(old_route)))
        return old_route

    fig = basemap(route, nearby_wmo=get_nearby_wmo(route), nearest_wmo=get_nearest_wmo(route),
                  title='Ogimet route algorithms', text_labels=('splits',), legend_only=('splits', 'nearby wmo'))
    fig.add_trace(
        scatter_geopoints(get_old_ogimet_route(),
                      mode='lines+markers',
                      name='ogimet route',
                      marker=dict(size=5, color='blue'),
                      opacity=1,
                      ))
    fig.add_trace(
        scatter_geopoints(get_new_ogimet_route(),
            mode='lines+markers',
            name='new ogimet route',
            marker=dict(size=5, color='red'),
            opacity=0.9,
        ))
    return VBox([go.FigureWidget(fig), log_view])

#plot()

interactive(children=(Dropdown(description='file', index=5, options=('ofp/BST_AF650 _LFPG_MMUN_20180304_1355z.…